# Introduction

Writing effective algorithms with Python requires comptence with the <span style="font-family:'Courier New'">numpy</span> package, which enables:
- Fast execution
- Minimum memory footprint

Understanding <span style="font-family:'Courier New'">numpy</span>, its speed, and how to use it more effectively requires a deeper examination of how variables are handled in memory.    

This Jupyter notebook covers the essential basis of <span style="font-family:'Courier New'">numpy</span> syntax and methods to get you on your way to writing faster code that uses less memory.  It focuses on basic techniques that are frequently useful in writing algorithms.

# <span style="font-family:'Courier New'">numpy</span> versus Python Lists

The first thing to understand is that <code>numpy</code> derives its speed from storing elements of arrays in contiguous blocks of memory and relying on the fast C programming language.

![contiguous_memory](images/numpy_vs_list.jpg)

While the advantages of faster execution with <code>numpy</code> are enormous a downside is that changing an array often causes it to be re-instantiated because, in part, the current contiguous memory block is no longer appropriate and so a new block of memory needs to be found and set up.

# Speed: Establish <span style="font-family:'Courier New'">numpy</span> Variables Once <a class="anchor" id="instan_numpy-once">

Any of these actions cause a <code>numpy</code> array to be re-instantiated, which takes a non-neglible amount of time:
    
- Concatenating <span style="font-family:'Courier New'">numpy</span> arrays with <code>np.concatenate()</code>
- Appending values to <span style="font-family:'Courier New'">numpy</span> arrays with <code>np.append()</code>
- Using <code>numpy np.vstack()</code> or <code>np.hstack()</code>
- Change the data type of a <span style="font-family:'Courier New'">numpy</span> array with <code>ndarray.astype()</code>
- Resize a <span style="font-family:'Courier New'">numpy</span> array with <code>np.resize()</code>

Performing any one of these operations once on an array is fine.  Do not, however, perform any of these commands within a loop so that these operations are repeated many times.  Applying these commands multiple times within a loop is a poor idea: find a better, faster way.

Alternate approaches to constructing a <code>numpy</code> array for which all the data is not immediately available include these:

- Determine the required _size_, _shape_, and _data type_ of an array and establish it once by using either <code>np.zeros()<code> or <code>np.empty()<code>.  (The former is usually the better choice.)  Then, fill the reserved space with values are they are created.

- Accumulate data with a Python list (or list of lists) and then, when all data is accumulated, convert the data once to a <code>numpy</code> array.

Assume in the example below that we are filling a <span style="font-family:'Courier New'">numpy</span> array with computed values, which I will simulate with random numbers.  The cells below illustrate methods that are slow because of repeated use of <code>numpy</code> array reinstantiation and faster methods along the lines of the bullets immediately above.  

In [1]:
import numpy as np
import time

In [2]:
nrows = 10000
ncols = 10

In [3]:
start = time.time()
np_arr = np.random.rand(1,ncols)
for i in range(nrows-1):
    np_arr = np.vstack((np_arr, np.random.rand(1,ncols)))
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_arr.shape)

Execution time with reserved ndarray: 0.22185564041137695
(10000, 10)


In [4]:
start = time.time()
np_arr = np.random.rand(ncols,1)
for i in range(nrows-1):
    np_arr = np.hstack((np_arr, np.random.rand(ncols,1)))
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_arr.shape)

Execution time with reserved ndarray: 0.23386001586914062
(10, 10000)


In [9]:
start = time.time()
np_arr = np.random.rand(1,ncols)
for i in range(nrows-1):
    np_arr = np.append(np_arr, np.random.rand(1,ncols), axis=0)
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_arr.shape)

Execution time with reserved ndarray: 0.2251126766204834
(10000, 10)


In [8]:
start = time.time()
np_arr = np.random.rand(1,ncols)
for i in range(nrows-1):
    np_arr = np.concatenate((np_arr, np.random.rand(1,ncols)))
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_arr.shape)

Execution time with reserved ndarray: 0.2072908878326416
(10000, 10)


It is much faster to create a numpy array of sufficient size once to reserve space and just replace its values as the algorithm progresses.

In [11]:
np_res = np.zeros((nrows, ncols))

In [12]:
start = time.time()
for i in range(nrows):
    new_row = np.random.rand(1,ncols)
    np_res[i] = new_row
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_res.shape)

Execution time with reserved ndarray: 0.01495671272277832
(10000, 10)


If you do feel the need to append to a data structure as an algorithm progresses without defining a <code>numpy</code> array to reserve space, then accumulating data initially in a Python list before converting to a <code>numpy</code> is much faster.

In [14]:
start = time.time()
np_arr = []
for i in range(nrows):
    np_arr.append(np.random.rand(1,ncols)) 
np_arr = np.array(np_arr)
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_arr.shape)

Execution time with reserved ndarray: 0.018949031829833984
(10000, 1, 10)


## Avoid Loops with <code>numpy</code> and Elementwise Calculations (Vectorization)

As we have discussed, your code slows dramatically with each nested <code>for</code> loop you add.  You can avoid using loops with <code>numpy</code> vectorization.  While the loops are eliminated from your Python code, a looping mechanism is still executed behind the scenes in <code>numpy</code>, although <code>numpy</code> does this operation much more quickly than if it was done with Python code.

### The Traditional Python Approach to Array Addition with Loops

In [ ]:
x = [[0,1,2],[3,4,5],[6,7,8]]
y = [[1,1,1],[1,1,1],[1,1,1]]
z = [[0,0,0],[0,0,0],[0,0,0]]

for i in range(len(x)):
    for j in range(len(x[0])):
        z[i][j] = x[i][j] + y[i][j]
print(z)

### Array Addition With <code>numpy</code>, Without loops

In [ ]:
x = np.array([[0,1,2],[3,4,5],[6,7,8]])
y = np.array([[1,1,1],[1,1,1],[1,1,1]])

z = x + y
print(z)

### A Bigger Addition Problem

In [ ]:
import random
prob_size = 1000
x = [[random.randint(0,10) for _ in range(prob_size)] for _ in range(prob_size)]
y = [[random.randint(0,10) for _ in range(prob_size)] for _ in range(prob_size)]
z = [[0 for _ in range(prob_size)] for _ in range(prob_size)]

time_start = time.time()
for i in range(len(x)):
    for j in range(len(x[0])):
        z[i][j] = x[i][j] + y[i][j]
print(f'for loop execution time: {time.time() - time_start}')

x = np.random.randint((prob_size,prob_size))
y = np.random.randint((prob_size,prob_size))

time_start = time.time()
z = x + y
print(f'numpy execution time: {time.time() - time_start}')

## Selecting Elements from <code>numpy</code> Arrays 

Before we continue with the topic of writing faster code, let's refresh or learn about some very useful <code>numpy</code> methods.

- <span style="font-family:'Courier New'">np.min()</span>
- <span style="font-family:'Courier New'">np.max()</span>
- <span style="font-family:'Courier New'">np.argmin()</span>
- <span style="font-family:'Courier New'">np.argmax()</span>

Algorithms frequently require that either the minimum or maximum elements be selected from an array/list or, in a more complex manner, the best element fitting particular criteria is sought.

One one just find the least or greatest array elements using the <code>np.min()</code> or <code>np.max()</code> methods, respectively.

In [15]:
x = np.random.randint(0,10,(10,))
x

array([7, 8, 0, 8, 4, 2, 0, 2, 3, 4])

In [16]:
print(x)
print(x.min(), np.min(x))
print(x.max(), np.max(x))

[7 8 0 8 4 2 0 2 3 4]
0 0
8 8


One might also find the leat and greatest elements using the <code>np.argmin()</code> or <code>np.argmax()</code> methods, respectively, although this requires a second statement to actually retrieve the element values.

In [17]:
idx_min = x.argmin()
idx_max = x.argmax()
print(x)
print(idx_min, x[idx_min])
print(idx_max, x[idx_max])

[7 8 0 8 4 2 0 2 3 4]
2 0
1 8


Despite needing a second statement to obtain a value, knowing the index of a minimum/maximum is quite useful when one must select multiple elements from an array and keep track of which elements have been selected so that they are not selected again.  This is the focus of a subsequent section in this Jupyter notebook.

The <code>np.argsort()</code> method can be useful to find the element from a list that, rather than being the least or greatest element, is the largest (smallest) item smaller (larger) than some upper (lower)limit.

In [ ]:
idx_sort = np.argsort(x)
print(f'x: {x}')
print(f'idx_sort: {idx_sort}')
print(f'x[idx_sort]: {x[idx_sort]}')

In [ ]:
# Find the largest element less than 5
i = -1
while x[idx_sort[i+1]] < 5 and i+1 < x.shape[0]:
    i += 1
print(i, idx_sort[i], x[idx_sort[i]])

Recall one method for sorting in descending order.

In [ ]:
idx_sort = np.argsort(x)
idx_sort = np.flip(idx_sort)
print(f'x: {x}')
print(f'idx_sort: {idx_sort}')
print(f'x[idx_sort]: {x[idx_sort]}')

This is another method, although it is perhaps less intuitive.

In [ ]:
idx_sort = np.argsort(x)
idx_sort = idx_sort[::-1]
print(f'x: {x}')
print(f'idx_sort: {idx_sort}')
print(f'x[idx_sort]: {x[idx_sort]}')

## Application: The Cell Tower Problem

In [ ]:
import random

def setup():
    prob_size = 100000
    data = [random.random() for _ in range(prob_size)]
    budget = 5.0
    return data, budget

### Python List with Deletion of Used Elements

### With a <code>for</code> Loop

In [ ]:
import random
import time

towers, budget = setup()
time_start = time.time()

towers_to_pick = []

for t in towers:
    if sum(towers_to_pick) + t <= budget:
        towers_to_pick.append(towers[0])
    del towers[0]

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick}')

In [ ]:
import random
import time

towers, budget = setup()
time_start = time.time()

towers_to_pick = []

for t in towers:
    if sum(towers_to_pick) + t <= budget:
        towers_to_pick.append(towers.pop(0))
    else:
        _ = towers.pop(0)

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick}')

### With a <code>while</code> Loop

Using a <code>while</code> loop in this situation in a way to iterate through all the data is an inferior method to using a <code>for</code> loop, but the examples are shown for speed comparison in any case.

In [ ]:
import random
import time

towers, budget = setup()
time_start = time.time()

towers_to_pick = []

while sum(towers_to_pick) < budget and len(towers) > 0:
    if sum(towers_to_pick) + towers[0] <= budget:
        towers_to_pick.append(towers.pop(0))
    else:
        _ = towers.pop(0)

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick}')

In [ ]:
import random
import time

towers, budget = setup()
time_start = time.time()

towers_to_pick = []
while sum(towers_to_pick) < budget and len(towers) > 0:
    if sum(towers_to_pick) + towers[0] <= budget:
        towers_to_pick.append(towers[0])
    del towers[0]
    # towers = towers[1:]

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick}')

### <code>numpy</code> with Slices to Delete Used Elements

In [ ]:
import numpy as np

towers, budget = setup()
towers = np.array(towers)
time_start = time.time()

towers_to_pick = np.array([])

while towers_to_pick.sum() < budget and towers.shape[0] > 0:
    if towers_to_pick.sum() + towers[0] <= budget:
        towers_to_pick = np.append(towers_to_pick, towers[0])
    towers = towers[1:]

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick}')

### Efficient <code>numpy</code> with Reserved Memory for Array

In [ ]:
import numpy as np

towers, budget = setup()
towers = np.array(towers)
time_start = time.time()

''' Reserve space for solution of maximum possible size '''
towers_to_pick = np.zeros(towers.shape[0], dtype=np.float32)  # do not use np.empty()!!!

j = 0  # counter for number of elements packed and the index of the next element to be packed
for vol in towers:
    if vol <= budget:
        towers_to_pick[j] = vol
        budget -= vol
        j += 1

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick[:j]}')

## Selecting Elements from <code>numpy</code> Arrays 

Before we continue with the topic of writing faster code, let's refresh or learn about some very useful <code>numpy</code> methods.

- <span style="font-family:'Courier New'">np.min()</span>
- <span style="font-family:'Courier New'">np.max()</span>
- <span style="font-family:'Courier New'">np.argmin()</span>
- <span style="font-family:'Courier New'">np.argmax()</span>

Algorithms frequently require that either the minimum or maximum elements be selected from an array/list or, in a more complex manner, the best element fitting particular criteria is sought.

One one just find the least or greatest array elements using the <code>np.min()</code> or <code>np.max()</code> methods, respectively.

In [ ]:
x = np.random.randint(0,10,(10,))

In [ ]:
print(x)
print(x.min(), np.min(x))
print(x.max(), np.max(x))

One might also find the leat and greatest elements using the <code>np.argmin()</code> or <code>np.argmax()</code> methods, respectively, although this requires a second statement to actually retrieve the element values.

In [ ]:
idx_min = x.argmin()
idx_max = x.argmax()
print(x)
print(idx_min, x[idx_min])
print(idx_max, x[idx_max])

Despite needing a second statement to obtain a value, knowing the index of a minimum/maximum is quite useful when one must select multiple elements from an array and keep track of which elements have been selected so that they are not selected again.  This is the focus of a subsequent section in this Jupyter notebook.

The <code>np.argsort()</code> method can be useful to find the element from a list that, rather than being the least or greatest element, is the largest (smallest) item smaller (larger) than some upper (lower)limit.

In [ ]:
idx_sort = np.argsort(x)
print(f'x: {x}')
print(f'idx_sort: {idx_sort}')
print(f'x[idx_sort]: {x[idx_sort]}')

In [ ]:
# Find the largest element less than 5
i = -1
while x[idx_sort[i+1]] < 5 and i+1 < x.shape[0]:
    i += 1
print(i, idx_sort[i], x[idx_sort[i]])

Recall one method for sorting in descending order.

In [ ]:
idx_sort = np.argsort(x)
idx_sort = np.flip(idx_sort)
print(f'x: {x}')
print(f'idx_sort: {idx_sort}')
print(f'x[idx_sort]: {x[idx_sort]}')

This is another method, although it is perhaps less intuitive.

In [ ]:
idx_sort = np.argsort(x)
idx_sort = idx_sort[::-1]
print(f'x: {x}')
print(f'idx_sort: {idx_sort}')
print(f'x[idx_sort]: {x[idx_sort]}')

## Boolean Masks

A Boolean (<code>True</code>/<code>False</code>) array can be used to filter out values from a <code>numpy</code> array.  Array elements whose position coincide with a <code>False</code> are filtered out.

### Example 1

In [ ]:
size = 5
x = np.arange(size)
x

In [ ]:
mask_x = np.array([True if i%2==1 else False for i in range(size)])
mask_x

In [ ]:
print(x[mask_x])

### Example 2

In [ ]:
y = np.arange(size**2).reshape(size,size)
y

In [ ]:
mask_y = np.array([True if i%2==1 else False for i in range(size**2)]).reshape(5,5)
mask_y

In [ ]:
print(y[mask_y])

### Example 3: Select Array Rows

In [ ]:
row_mask = [False, True, False, True, True]

In [ ]:
y[row_mask]

In [ ]:
y[row_mask,:]

### Example 4: Select Array Columns

In [ ]:
col_mask = [True, False, False, False, True]

In [ ]:
y[:,col_mask]

### Example 5: Select Array Rows and columns

In [ ]:
y[row_mask,:][:,col_mask]

### Example 6: Select on Criterion

In [ ]:
z = np.random.random(size = (size,))
z

In [ ]:
z >= 0.5

In [ ]:
mask_z = (z >= 0.5)
mask_z

In [ ]:
z[mask_z]

## Application 2: Traveling Salesperson Problem

In this problem, the task is to maintain the original data in its original instantiation without deleting the data pertaining to the destinations already included in the Traveling Salesperson's route.

### A Traveling Salesperson Problem (TSP) Greedy Algorithm

Randomly select a location to start.  Assume that we select Location 1.

- Loop until all locations visited
  - For each location, choose the next location to be closest possible next location of locations not yet visited
  
![AlgoStep1](images/m1.jpg)
![AlgoStep2](images/m2.jpg)
![AlgoStep3](images/m3.jpg)
![AlgoStep4](images/m4.jpg)
![AlgoStep5](images/m5.jpg)

Route: 1-2-0-4-3-1

This algorithm could be implemented by deleting an array column when each next stop location is determined.  One could, alternately, "mask" out those columns so that locations already in the route could not be revisited.  The latter approach avoids needing to re-instantiate the array multiple times.

#### Set up the data

In [ ]:
# create distance matrix
nloc = 10
dist = np.random.rand(nloc,nloc)
dist = np.triu(dist,k=0)
for i in range(1,nloc):
    for j in range(0, i):
        dist[i,j] = dist[j,i]
for i in range(nloc):
    dist[i,i]=0.0
dist

In [ ]:
''' Set up parameters '''
nloc = dist.shape[0]                      # number of locations
assert dist.shape[0] == dist.shape[1]     # ensure square distance matrix

''' Initialize random starting point '''
start = np.random.randint(0, nloc-1)      # select random starting location
sol = [start]                             # solution route in a list
cur_loc = start                           # use cur_loc to indicate current location index

''' Establish Boolean mask for the columns: True = column location not visited '''
col_mask = np.ones(nloc).astype(np.bool_) # creates array of True
col_mask[start] = False                   # cannot choose starting location as
                                          # next location

''' Create ndarray of column indices '''
col_indices = np.arange(nloc)             # create array of indices

''' Initial distance of solution '''
sol_dist = 0.0                            # initialize distance of solution

''' Execute algorithm '''
while col_mask.sum() > 0:              # continue if any True values in col_mask
    ''' Get index of next location '''
    next_loc_ind = np.argmin(dist[cur_loc][col_mask])  # get index of row minimum for
                                                       #  remaining locations
    next_loc_ind = col_indices[col_mask][next_loc_ind] # find index of minimum relative to original
                                                       #   indices (true index of location)
    
    ''' Update solution and mask '''
    sol.append(next_loc_ind)                   # append next location to solution
    col_mask[next_loc_ind] = False             # update mask for current location
    sol_dist += dist[cur_loc, next_loc_ind]    # update solution distance
    cur_loc = next_loc_ind                     # update current location

sol.append(start)       # append starting location for round trip
sol, sol_dist